In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pickle

In [8]:
data = pd.read_csv('./Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
## Preprocess the data
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [19]:
for col in data.columns:
    print(data[col].value_counts())

CreditScore
850    233
678     63
655     54
705     53
667     53
      ... 
404      1
351      1
365      1
417      1
419      1
Name: count, Length: 460, dtype: int64
Gender
1    5457
0    4543
Name: count, dtype: int64
Age
37    478
38    477
35    474
36    456
34    447
     ... 
92      2
82      1
88      1
85      1
83      1
Name: count, Length: 70, dtype: int64
Tenure
2     1048
1     1035
7     1028
8     1025
5     1012
3     1009
4      989
9      984
6      967
10     490
0      413
Name: count, dtype: int64
Balance
0.00         3617
130170.82       2
105473.74       2
85304.27        1
159397.75       1
             ... 
81556.89        1
112687.69       1
108698.96       1
238387.56       1
130142.79       1
Name: count, Length: 6382, dtype: int64
NumOfProducts
1    5084
2    4590
3     266
4      60
Name: count, dtype: int64
HasCrCard
1    7055
0    2945
Name: count, dtype: int64
IsActiveMember
1    5151
0    4849
Name: count, dtype: int64
EstimatedSalary
24924.92  

In [10]:
## Encoding categorical featres
label_encoder_gender = LabelEncoder()
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')

data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

encoded_geo = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
encoded_geo_df = pd.DataFrame(encoded_geo, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [11]:
## Combine Geography encoded dataframe to original dataframe
data = pd.concat([data.reset_index(drop=True), encoded_geo_df], axis=1)
data = data.drop('Geography', axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [13]:
## Split data into dependent and independent features
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [22]:
## Train Test Split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

## Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [23]:
# Save all pickle files
with open('label_encoder_gender0.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo0.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler0.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [24]:
## ANN Regression problem statement
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), #HL1
    Dense(32, activation='relu'), ##HL2
    Dense(1) ## Output Layer, default activation function is linear activation function
])

## compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

## Set up tensorboard
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callbacks = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [26]:
## Setup Early stopping
early_stopping_callbacks = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [27]:
## Train the model
history = model.fit(
    X_train, y_train,
    validation_data = [X_test, y_test],
    epochs=100,
    callbacks=[early_stopping_callbacks, tensorboard_callbacks]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 100196.7031 - mae: 100196.7031 - val_loss: 99270.5703 - val_mae: 99270.5703
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 99456.9766 - mae: 99456.9766 - val_loss: 97728.0781 - val_mae: 97728.0781
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 96765.8594 - mae: 96765.8594 - val_loss: 93705.7578 - val_mae: 93705.7578
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 91429.3672 - mae: 91429.3672 - val_loss: 87057.6016 - val_mae: 87057.6016
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 83675.6875 - mae: 83675.6875 - val_loss: 78409.7891 - val_mae: 78409.7891
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 74601.0703 - mae: 74601.0703 - val_loss: 69552.0938 - val_mae: 69552.0938
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 65779.7578 - mae: 65779.7578 

In [1]:
%load_ext tensorboard 

In [4]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 31920), started 19:37:20 ago. (Use '!kill 31920' to kill it.)

In [47]:
## Evaluate model on test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

63/63 [==============================] - 0s 933us/step - loss: 50743.5195 - mae: 50743.5195
Test MAE: 50743.51953125


In [48]:
## save model file
model.save('regression_model.h5')

c:\NLP_PROJECTS\ANN_classification_churn\ANN_classification_churn\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
